In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
    
class HTMLTableParser:
       
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  
    
    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []
    
        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
                
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                        
            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
    
        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")
    
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                    
        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
            
        return df

In [2]:
hp = HTMLTableParser()
url = 'https://www.quechoisir.org/comparatif-additifs-alimentaires-n56877/?f4%5B%5D=EQpicto-3-orange.png&f4%5B%5D=EQpicto-4-rouge.png#filtres'
additifs_df = hp.parse_url(url)[0][1]
additifs_df

,,Produit,Fonction principale,Notre appréciation
0,\n\n,\n\n\nE102Tartrazine\nColorant azoïque \n\nFic...,Colorant,
1,\n\n,\n\n\nE104Jaune de quinoléine\nColorant azoïqu...,Colorant,
2,\n\n,"\n\n\nE110Jaune soleil FCF, Jaune orangé S\nCo...",Colorant,
3,\n\n,"\n\n\nE120Acide carminique, Carmins\n\nFiche P...",Colorant,
4,\n\n,"\n\n\nE122Azorubine, Carmoisine\nColorant azoï...",Colorant,
...,...,...,...,...
86,\n\n,"\n\n\nE1412Amidon modifié, phosphate de diamid...",Agent de texture,
87,\n\n,"\n\n\nE1413Amidon modifié, phosphate de diamid...",Agent de texture,
88,\n\n,"\n\n\nE1414Amidon modifié, phosphate de diamid...",Agent de texture,
89,\n\n,"\n\n\nE1442Amidon modifié, phosphate de diamid...",Agent de texture,


In [3]:
additifs_df.drop(['','Notre appréciation'],axis=1, inplace=True)

In [4]:
additifs_df.head()

,Produit,Fonction principale
0,\n\n\nE102Tartrazine\nColorant azoïque \n\nFic...,Colorant
1,\n\n\nE104Jaune de quinoléine\nColorant azoïqu...,Colorant
2,"\n\n\nE110Jaune soleil FCF, Jaune orangé S\nCo...",Colorant
3,"\n\n\nE120Acide carminique, Carmins\n\nFiche P...",Colorant
4,"\n\n\nE122Azorubine, Carmoisine\nColorant azoï...",Colorant


In [5]:
additifs_df['Produit'] = additifs_df['Produit'].str.replace('\n','')
additifs_df['Produit'] = additifs_df['Produit'].str.replace('E','e')

In [6]:
additifs_df.head()

,Produit,Fonction principale
0,e102TartrazineColorant azoïque Fiche Produit,Colorant
1,e104Jaune de quinoléineColorant azoïque Fiche ...,Colorant
2,"e110Jaune soleil FCF, Jaune orangé SColorant a...",Colorant
3,"e120Acide carminique, CarminsFiche Produit",Colorant
4,"e122Azorubine, CarmoisineColorant azoïque Fich...",Colorant


In [7]:
for i in additifs_df['Produit'].index:
    val = str(additifs_df.loc[i,'Produit'])
    additifs_df.loc[i,'Produit'] = val[:4]

In [8]:
additifs_df.head()

,Produit,Fonction principale
0,e102,Colorant
1,e104,Colorant
2,e110,Colorant
3,e120,Colorant
4,e122,Colorant


In [9]:
additifs_df.loc[(additifs_df['Produit'] == 'e141') | (additifs_df['Produit'] == 'e144') | (additifs_df['Produit'] == 'e145'), :]

,Produit,Fonction principale
85,e141,Agent de texture
86,e141,Agent de texture
87,e141,Agent de texture
88,e141,Agent de texture
89,e144,Agent de texture
90,e145,Agent de texture


In [10]:
additifs_df.loc[85, 'Produit'] = 'e1410'
additifs_df.loc[86, 'Produit'] = 'e1412'
additifs_df.loc[87, 'Produit'] = 'e1413'
additifs_df.loc[88, 'Produit'] = 'e1414'
additifs_df.loc[89, 'Produit'] = 'e1442'
additifs_df.loc[90, 'Produit'] = 'e1452'

In [11]:
additifs_df.tail()

,Produit,Fonction principale
86,e1412,Agent de texture
87,e1413,Agent de texture
88,e1414,Agent de texture
89,e1442,Agent de texture
90,e1452,Agent de texture


In [12]:
additifs_df.loc[(additifs_df['Produit'] == 'e472'), :]

,Produit,Fonction principale
43,e472,Agent de texture
44,e472,Agent de texture
45,e472,Agent de texture
46,e472,Agent de texture
47,e472,Agent de texture
48,e472,Agent de texture


In [13]:
liste = ['a','b','c','d','e','f']
for i in range(len(additifs_df.loc[(additifs_df['Produit'] == 'e472'), :])):
    additifs_df.loc[43+i, 'Produit'] = additifs_df.loc[43+i, 'Produit'] + liste[i] 

In [14]:
additifs_df.loc[43, :]

Produit                           e472a
Fonction principale    Agent de texture
Name: 43, dtype: object

In [15]:
import pickle

with open("transfer_additifs_df","wb") as fichier:
    additifs_df_pickler=pickle.Pickler(fichier)
    additifs_df_pickler.dump(additifs_df)